
# Privacy-Aware Remapping Mechanism Demo


This notebook provides a tour of how to replicate results obtained with the Privacy-Aware Remapping Mechanism in the Privkit Framework, including how to:
- load data
- apply the Planar Laplace mechanism
- apply Uniform Grid discretization
- apply the Privacy-Aware Remapping Mechanism
- obtain the quality loss
- obtain the number of utilized cells
- apply the Top-N Re-Identification Attack


## Abstract

In an era dominated by Location-Based Services (LBSs), the concern of preserving location privacy has emerged as a critical challenge. To address this, Location Privacy-Preserving Mechanisms (LPPMs) were proposed, in where an obfuscated version of the exact user location is reported instead. Adding to noise-based mechanisms, location discretization, the process of transforming continuous location data into discrete representations, is relevant for the efficient storage of data, simplifying the process of manipulating the information in a digital system and reducing the computational overhead. Apart from enabling a more efficient data storage and processing, location discretization can also be performed with privacy requirements, so as to ensure discretization while providing privacy benefits. In this work, we propose a Privacy-Aware Remapping mechanism that is able to improve the privacy level attained by Geo-Indistinguishability through a tailored pre-processing discretization step. The proposed remapping technique is capable of reducing the re-identification risk of locations under Geo-Indistinguishability, with limited impact on quality loss.

## Citation

Please consider to cite our publication in your scientific work:

In [ ]:
@inproceedings{10.1145/3605098.3636050,
    author = {Duarte, Guilherme and Cunha, Mariana and Vilela, Jo\~{a}o},
    title = {A Privacy-Aware Remapping Mechanism for Location Data},
    year = {2024},
    isbn = {9798400702433},
    publisher = {Association for Computing Machinery},
    address = {New York, NY, USA},
    url = {https://doi.org/10.1145/3605098.3636050},
    doi = {10.1145/3605098.3636050},
    booktitle = {Proceedings of the 39th ACM/SIGAPP Symposium on Applied Computing},
    pages = {1433–1440},
    numpages = {8},
    location = {<conf-loc>, <city>Avila</city>, <country>Spain</country>, </conf-loc>},
    series = {SAC '24}
}

----------------------

## Import Privkit

In [ ]:
import privkit as pk
pk.__version__

# Supress pandas warnings
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Loading Data

The dataset used for testing the mechanism was [Geolife](https://www.microsoft.com/en-us/research/publication/geolife-gps-trajectory-dataset-user-guide/) which contains reports from Beijing, China.

In [ ]:
dataset = pk.GeolifeDataset()
dataset.load_dataset()

dataset = dataset.data

## Uniform Grid Discretization
During testing, we limited the distribution of reports to a bounding box over 5<sup>th</sup> ring road, defined by the following parameters.
Additionally, for grid discretization, we set as constant spacing of the cell's grid the value of 100 meters.

In [ ]:
from privkit.utils import constants

# Parameters
min_lat = 39.753389
max_lat = 40.02577
min_lon = 116.198582
max_lon = 116.547398
spacing = 250

# Create Grid
dataset.create_grid(min_lat, max_lat, min_lon, max_lon, spacing)
# Filter outside points
dataset.filter_outside_points(min_lat, max_lat, min_lon, max_lon)

Once created the grid, we can discretize the dataset by replacing the original reports with the center of cell which where they belong. To achieve this, we rely on the UniformRemapping mechanism.

In [ ]:
from privkit.ppms import UniformRemapping

uniform_remapping = UniformRemapping(dataset.grid)
uniform_remapping.execute(dataset, data_processing=True)

discrete_ground_truth_data = dataset.data[[constants.LATITUDE, constants.LONGITUDE]]

## Applying the Planar Laplace Mechanism PPM
As a pre-processing step, apply Planar Laplace (PL) with a privacy parameter $\varepsilon$ to the dataset. During testing, we used multiple values in the typical ranges of privacy-preserving mechanisms for continuous reports, specifically $\varepsilon$ = [4, 8, 16, 32] km<sup>-1</sup>. The following code generates a dictionary which maps the privacy parameters to the obfuscated data resulted from the respective $\varepsilon$. This way we only one instance of the dataset loaded.

In [ ]:
epsilons = [0.004, 0.008, 0.016, 0.032]

planar_laplace_data = {}
for epsilon in epsilons:
    planar_laplace = pk.PlanarLaplace(epsilon=epsilon)
    planar_laplace.execute(dataset)
    planar_laplace_data[epsilon] = dataset.data[[constants.OBF_LATITUDE, constants.OBF_LONGITUDE]]
    print(f'{epsilon} done')

Similarly to before, let us discretize the obfuscated data into the grid, which corresponds to the Uniform Remapping:

In [ ]:
uniform_remapping_data = {}
for epsilon in epsilons:
    dataset.data[[constants.OBF_LATITUDE, constants.OBF_LONGITUDE]] = planar_laplace_data[epsilon]
    
    uniform_remapping = UniformRemapping(dataset.grid)
    uniform_remapping.execute(dataset, combined_ppm=True)

    uniform_remapping_data[epsilon] = dataset.data[[constants.OBF_LATITUDE, constants.OBF_LONGITUDE]]

## Applying the Privacy-Aware Remapping Mechanism
After applying the Planar Laplace PPM, we are ready to use Privacy-Aware Remapping. To configure this mechanism, one has to set the epsilon parameter and, optionally, the obfuscation radius parameter $r$, a bound on the Laplacian noise added, and the search space (the default is a 'circle' with radius r). By default, the Privacy-Aware Remapping consider $r=C_\varepsilon^{-1}(\rho) + s/\sqrt{2}$, where $\rho$ is an optional parameter (default $\rho=0.95$) and the cumulative densitity function $C_\varepsilon(r)$ represents the probability that the radius of the random generated point falls between 0 and $r$. Therefore, using the Lambert 𝑊 function at branch -1, the inverse function is defined as:

$$ C_\varepsilon^{-1}(\rho) = - \frac{1}{\varepsilon}(𝑊_{-1}(\frac{\rho+1}{e})+1)$$

Once again, we generate a dictionary which maps the privacy parameter to the obfuscation data resulted from the Privacy-Aware Remapping Mechanism with $\varepsilon$.

In [ ]:
import numpy as np

privacy_aware_remapping_data = {}
for epsilon in epsilons:
    dataset.data[[constants.OBF_LATITUDE, constants.OBF_LONGITUDE]] = uniform_remapping_data[epsilon]
    
    # Apply the mechanism
    privacy_aware_remapping = pk.PrivacyAwareRemapping(epsilon=epsilon, search_space='circle')
    privacy_aware_remapping.execute(dataset)
    
    privacy_aware_remapping_data[epsilon] = dataset.data[[constants.OBF_LATITUDE, constants.OBF_LONGITUDE]]
    print(f'{epsilon} done')

## Obtaining the Quality Loss
After applying the Privacy-Aware Remapping Mechanism, we can use the quality loss metric to measure how much utility was lost after using this technique, by calculating the Euclidean distance between every ground-truth report and its respective obfuscated report. For this test, we will compute the quality loss added by the Planar Laplace without Remapping, by the Planar Laplace with Remapping (Uniform Remapping) and by the Privacy-Aware Remapping Mechanism, storing the data on dictionaries that map $\varepsilon$ values to the list of point-by-point distances.

In [ ]:
planar_laplace_non_remapped_quality_loss = {}
uniform_remapping_quality_loss = {}
privacy_aware_remapping_quality_loss = {}

quality_loss = pk.QualityLoss()
for epsilon in epsilons:
    # Planar Laplace Non Remapped
    dataset.data[[constants.OBF_LATITUDE, constants.OBF_LONGITUDE]] = planar_laplace_data[epsilon]
    quality_loss.execute(dataset)
    planar_laplace_non_remapped_quality_loss[epsilon] = dataset.data[[quality_loss.METRIC_ID]]

    # Uniform Remapping
    dataset.data[[constants.OBF_LATITUDE, constants.OBF_LONGITUDE]] = uniform_remapping_data[epsilon]
    quality_loss.execute(dataset)
    uniform_remapping_quality_loss[epsilon] = dataset.data[[quality_loss.METRIC_ID]]

    # Privacy-Aware Remapping
    dataset.data[[constants.OBF_LATITUDE, constants.OBF_LONGITUDE]] = privacy_aware_remapping_data[epsilon]
    quality_loss.execute(dataset)
    privacy_aware_remapping_quality_loss[epsilon] = dataset.data[[quality_loss.METRIC_ID]]
    
    print(f'{epsilon} done')

Using the following program, one can visualize the amount of quality loss each mechanism is subject to.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

colors = ['r', 'g', 'b']
plt.subplots(1, len(epsilons), sharey=True, figsize=(6, 6))

for index, epsilon in enumerate(epsilons):
    plt.subplot(1, 4, index + 1)

    # Gather Data
    pl_ql_epsilon = planar_laplace_non_remapped_quality_loss[epsilon] 
    ur_ql_epsilon = uniform_remapping_quality_loss[epsilon]
    par_ql_epsilon = privacy_aware_remapping_quality_loss[epsilon]

    # Plot the Data
    pl_plot = plt.boxplot(pl_ql_epsilon, positions=[-0.3], widths=0.20, notch=True, patch_artist=True, showfliers=False)
    ur_plot = plt.boxplot(ur_ql_epsilon, positions=[0], widths=0.20, notch=True, patch_artist=True, showfliers=False)
    par_plot = plt.boxplot(par_ql_epsilon, positions=[0.3], widths=0.20, notch=True, patch_artist=True, showfliers=False)

    # Add Colors
    for plot, color in zip([pl_plot, ur_plot, par_plot], colors):
        for patch in plot['boxes']:
            patch.set_facecolor(color)

    # Settings
    plt.xticks(np.arange(0, 2, 2), [r'$\varepsilon={}$'.format(int(epsilon*1000))])
    plt.yticks(np.arange(0, 1300, 100))
    plt.axis(xmin=-2, xmax=2)
    plt.xlim([-0.6, 0.6])
    plt.ylim([0, 1200])
    plt.grid(True)

# Label
plt.subplot(1, 4, 1)
plt.ylabel('Quality Loss (m)')

# Legend
plt.subplot(1, 4, 4)
handles = []
for color, grid in zip(colors, ['Planar Laplace Non-Remapped', 'Uniform Remapping', 'Privacy-Aware Remapping']): 
    handles.append(mpatches.Patch(color=color, label=grid))
plt.legend(handles=handles, loc='upper right')

# Stick the four plots together
plt.subplots_adjust(wspace=.0)

# Show
plt.show()

## Obtain the Number of Utilized Cells
In addition to the quality loss, we also analyzed the effect on the number of utilized cells after using grid discretization on the ground-truth data, on the Planar Laplace obfuscated locations as well as using the Privacy-Aware Remapping mechanism.

In [ ]:
def utilized_cells(data, lat, lon):
    return len(set(zip(data[lat], data[lon])))

discrete_ground_truth_utilized_cells = utilized_cells(discrete_ground_truth_data, constants.LATITUDE, constants.LONGITUDE)
print(f'Ground Truth: {discrete_ground_truth_utilized_cells}')
for epsilon in epsilons:
    uniform_remapping_utilized_cells = utilized_cells(uniform_remapping_data[epsilon], constants.OBF_LATITUDE, constants.OBF_LONGITUDE)
    privacy_aware_remapping_utilized_cells = utilized_cells(privacy_aware_remapping_data[epsilon], constants.OBF_LATITUDE, constants.OBF_LONGITUDE)
    
    print(f'epsilon: {epsilon}')
    print(f'   Uniform Remapping: {uniform_remapping_utilized_cells}')
    print(f'   Privacy-Aware Remapping: {privacy_aware_remapping_utilized_cells}')

## Apply the Top-N Re-Identification Attack

Finally, we can test how the mechanisms behave against the Top-N Re-Identification Attack. For each instance of the attack, one must feed the paramenter $N$, the number of top-locations to considered. During testing, we analyzed top-1, top-2 and top-3 locations.

In [ ]:
Ns = [1,2,3]
ground_truth_remapped_topn_data = {}
uniform_remapping_topn_data = {}
privacy_aware_remapping_topn_data = {}

# Ground Truth Remapped
print('Ground Truth Remapped')
for N in Ns:
    print(f'   - N = {N}')
    dataset.data[[constants.LATITUDE, constants.LONGITUDE]] = discrete_ground_truth_data
    topN = pk.TopN(N=N, over_ppm=False)
    results = topN.execute(dataset)
    ground_truth_remapped_topn_data[N] = results

# PPMs
print('PPMs')
for epsilon in epsilons:
    print(f'   - eps = {epsilon}')
    uniform_remapping_topn_data[epsilon] = {}
    privacy_aware_remapping_topn_data[epsilon] = {}
    
    for N in Ns:
        print(f'      - N = {N}')
        # Uniform Remapping
        dataset.data[[constants.OBF_LATITUDE, constants.OBF_LONGITUDE]] = uniform_remapping_data[epsilon]
        topN = pk.TopN(N=N, over_ppm=True)
        results = topN.execute(dataset)
        uniform_remapping_topn_data[epsilon][N] = results
        
        # Privacy-Aware Remapping
        dataset.data[[constants.OBF_LATITUDE, constants.OBF_LONGITUDE]] = privacy_aware_remapping_data[epsilon]
        topN = pk.TopN(N=N, over_ppm=True)
        results = topN.execute(dataset)
        privacy_aware_remapping_topn_data[epsilon][N] = results

To visualize the obtained results, the following code generates a plotbar comparing the percentage of users from the dataset considered re-identifiable.

In [ ]:
fig, axes = plt.subplots(1, len(epsilons), sharey=True, figsize=(13, 6))
barWidth = 0.25
nr_users = len(dataset.data[constants.UID].unique())

for index, epsilon in enumerate(epsilons):
    plt.subplot(1, 4, index + 1)
    ax = axes[index]

    # Gather data
    gt_data = []
    ur_data = []
    par_data = []

    for N in Ns:
        gt_topn = ground_truth_remapped_topn_data[N]
        ur_topn = uniform_remapping_topn_data[epsilon][N]
        par_topn = privacy_aware_remapping_topn_data[epsilon][N]

        # Calculate ratio of re-identification
        gt_reident = (sum(gt_topn.values()) / nr_users) * 100
        ur_reident = (sum(ur_topn.values()) / nr_users) * 100
        par_reident = (sum(par_topn.values()) / nr_users) * 100

        gt_data.append(gt_reident)
        ur_data.append(ur_reident)
        par_data.append(par_reident)

    # Bar locations
    br1 = np.arange(len(Ns))
    br2 = [x + barWidth for x in br1]
    br3 = [x + barWidth for x in br2]

    # Create bars
    for br, data, color, label in zip([br1, br2, br3], [gt_data, ur_data, par_data], colors, ['Ground-Truth Remapped', 'Uniform Remapping', 'Privacy-Aware Remapping']):
        ax.bar(br, data, color=color, width=barWidth, edgecolor='grey', label=label)
    
    # Set scenario label
    ax.set_xlabel(r'$\epsilon={}$'.format(int(epsilon*1000)), fontsize=12)

    # Subplot Settings
    ax.set_xlim([-3*barWidth/2, br3[-1] + 3*barWidth/2])
    ax.set_ylim([-1, 100])
    ax.set_xticks([r + barWidth for r in br1], [f'N={N}' for N in Ns])
    ax.set_axisbelow(True)
    ax.grid(which='both')

# Settings
plt.subplot(1, 4, 1)
plt.yticks(np.arange(0, 101, 5))
plt.ylabel('Users re-identified (%)')

# Legend
ax = axes[1]
ax.legend(bbox_to_anchor=(1, -0.14), loc='center', ncol=3)

# Stick the four plots together
plt.subplots_adjust(wspace=.0)

# Show
plt.show()